In [143]:
from itertools import product
from functools import reduce
from random import random, randint, shuffle, seed, choice
import numpy as np
from scipy import sparse
from copy import copy

In [144]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [145]:
x = make_set_covering_problem(1000, 1000, .3)
print("Element at row=42 and column=42:", x[42, 42])

Element at row=42 and column=42: False


In [146]:
SETS = x.toarray()
PROBLEM_SIZE = NUM_SETS = 1000

def fitness(state): #I'll use fitness2 of set-covering_single_state
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return valid, -cost

In [147]:
def tweak(state): #I'll use the same tweak function
    new_state = copy(state)
    index = randint(0, PROBLEM_SIZE - 1)
    new_state[index] = not new_state[index]
    return new_state

In [148]:
def hillClimbing(initial_state, max_steps):
    current_state = initial_state
    best_state = current_state
    best_fitness, _ = fitness(current_state)

    for step in range(max_steps):
        # Generate a neighbor
        neighbor = tweak(current_state)
        neighbor_fitness, _ = fitness(neighbor)

        # If the neighbor is better, move to the neighbor
        if neighbor_fitness >= best_fitness:
            current_state = neighbor
            best_state = current_state
            best_fitness = neighbor_fitness
    print(step+1)
    return best_state, best_fitness

In [149]:
def hillClimbingModified(initial_state, max_steps): #I'll use a modified version of Hill Climbing
    current_state = initial_state
    best_state = current_state
    best_fitness, _ = fitness(current_state)

    for step in range(max_steps):
        # Generate a neighbor
        neighbor = tweak(current_state)
        neighbor_fitness, _ = fitness(neighbor)

        # If the neighbor is better, move to the neighbor
        if neighbor_fitness >= best_fitness:
            current_state = neighbor
            best_state = current_state
            best_fitness = neighbor_fitness
        
        if fitness(neighbor) >= fitness(current_state):
            current_state = neighbor
            
            if fitness(current_state)[0] == PROBLEM_SIZE: #If I'll have all the tiles covered, I'll break the iterations
                print(step+1)
                break
    return best_state, best_fitness

In [150]:
initial_state = [choice([False, False, False, False, False, False]) for _ in range(NUM_SETS)]
best_state, best_fitness = hillClimbing(initial_state, 10000)

print("Hill CLimbing:")
print("Best State:", best_state)
print("Best Fitness:", fitness(best_state))

best_state, best_fitness = hillClimbingModified(initial_state, 10000)
print("Hill CLimbing Modified:")
print("Best State:", best_state)
print("Best Fitness:", fitness(best_state))

10000
Hill CLimbing:
Best State: [False, True, False, True, False, True, True, False, False, True, False, False, True, False, False, False, True, True, True, True, False, True, True, True, True, False, True, True, True, False, True, True, True, True, False, False, True, False, True, True, False, True, True, True, True, True, False, False, False, True, True, False, True, True, True, False, False, False, False, False, False, False, True, False, False, True, True, False, True, True, False, False, True, True, False, True, True, False, True, True, True, True, True, True, False, False, True, True, True, False, True, False, False, False, False, False, False, True, True, True, True, True, True, True, False, True, True, True, False, False, False, True, True, True, True, False, True, False, True, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, True, True, True, False, False, True, False, True, False, True, False, False, True, True, False, Tr